# 1. Load Libraries

In [1]:
import time
import numpy as np
import pandas as pd
from torchtext import vocab ## for glove vectors
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
# from torchtext.data import TabularDataset
from nltk import word_tokenize

In [2]:
print(torch.__version__)

2.0.0+cpu


In [3]:
train = pd.read_csv(r'/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
# validation = pd.read_csv(r'/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')

In [4]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
## Take small sample 
train = train[:1000]
train2 = train.copy()
# validation = validation[:1000]

# 2. Preprocessing

### A. train data

In [5]:
## preprocessing 
import nltk 
from nltk.corpus import stopwords

stopword = stopwords.words('english')

# define a function to remove stopwords
def remove_stopwords(txt):
    filtered_word = []
    words = txt.split(' ')
    for word in words:
        if word not in stopword:
            filtered_word.append(word)
    return ' '.join(filtered_word)

## clean text remove all special characters
import re, string

def clean_text(txt):
    txt = str(txt).lower()
    txt = re.sub('\[.*?\]','',txt)
    txt = re.sub('https?://\S+|www\.\S+','',txt)
    txt = re.sub('<.*?>+','',txt)
    txt =re.sub('[%s]' % re.escape(string.punctuation),'',txt)
    txt = re.sub('\n','',txt)
    txt = re.sub('\w*\d\w*','',txt)
    return txt

In [6]:
train['comment_text'] = train['comment_text'].apply(clean_text)
train['comment_text'] = train['comment_text'].apply(remove_stopwords)

# train.to_csv('/kaggle/working/' + 'train.csv')

In [7]:
## Load Data into DataLoader

## B. Validation Data



In [8]:
# ## we need to get the stopwords for all five languagee
# #append all the other languages stopwords

# tuk_stopwords = stopwords.words('turkish')
# rus_stopwords = stopwords.words('russian')
# ital_stopwords = stopwords.words('italian')
# french_stopwords = stopwords.words('french')
# portugese_stopwords = stopwords.words('portuguese')
# spanish_stopwords = stopwords.words('spanish')

# test_stopwords = tuk_stopwords + rus_stopwords + ital_stopwords + french_stopwords + portugese_stopwords + spanish_stopwords

# ## define a funct to remove stopwords from test data
# def test_stopword(txt):
#     words = txt.split(' ')
#     filtered_word = []
#     for word in words:
#         if word not in test_stopwords:
#             filtered_word.append(word)
#     return ' '.join(filtered_word)

# ## remove stopwords from test data 
# # test['content'] = test['content'].apply(test_stopword)
# # test['content'] = test['content'].apply(clean_text)

In [9]:
# ## validation stopwords
# validation_stopword = tuk_stopwords + ital_stopwords + spanish_stopwords

# def valid_stop(txt):
#     words = txt.split(' ')
#     filtered_word = []
#     for word in words:
#         if word not in validation_stopword:
#             filtered_word.append(word)
#     return ' '.join(filtered_word)

# validation['comment_text'] = validation['comment_text'].apply(valid_stop)
# validation['comment_text'] = validation['comment_text'].apply(clean_text)

# validation.head()

In [10]:
# #Approach 1 
# # tokenize the comment_text,
# ## use toxic as label 
# ## convert to tensor 
# ## 

# from nltk import word_tokenize

# # tokenize function
# def tokenize_text(txt):
#     txt = txt.lower()
#     tokens = []
#     words = txt.split(' ')
#     for word in words:
#         token = word_tokenize(word)
#         tokens.append(token)
#     return tokens


# ## use train['comment_text'] 
# ## get all the words 
# ## get  unique words  create vocab (tokenization is done of unique words)
# ## tokenize them 
# ## use those to create index 
# ## 

# ## Builduing vocabulary 
# # count the words

# tokenized_texts = train['comment_text'].apply(tokenize_text)
        
    


In [11]:
# ## create a library of unique_words
# unique_word = []
# for i, j in enumerate(train['comment_text']):
#     words = j.split(' ')
#     for word in words:
#         if word not in unique_words:
#             unique_word.append(word)

In [12]:
# from nltk.tokenize import word_tokenize

# #1. tokenize each comment
# tokenized_text = []
# for i,j in enumerate(train['comment_text']):
#         word = j.lower()
#         x = word_tokenize(word)
#         tokenized_text.append(x)
            
            
# #2. Get unique words
# unique_word = set()

# for sentence in tokenized_text:
#     for word in sentence:
#         unique_word.add(word)
        
                       
# # 3. sort alphabetically 
# unique_word = list[unique_word]
# unique_word = sorted(unique_word)

# #4. Assign id to each token
# word_to_id = {}
# for id, word in enumerate(unique_word):
#     word_to_id[word] = id

# #6. Assign id to word
# id_to_word = {}
# for token,id in word_to_id.items():
#     id_to_word[id] =token
    
# #7. Add padding to the snetences which has less length




In [13]:
# train['comment_text']

## 3. Convert the text to tokenize for the model 

In [14]:
#define tokenizer
def tokenizer(txt):
    word = word_tokenize(txt.lower())
    return word

#tokenize sentences
train['comment_text'] = train['comment_text'].apply(tokenizer)

# print(train['comment_text'])

unique_word = set()
for sentence in train['comment_text']:
    for word in sentence:
        unique_word.add(word)
        
#convert unique words to list
unique_word = list(unique_word)

# print(unique_word)
#create a dictionary of indexes of tokens
word_to_idx = {}
for idx,word in enumerate(sorted(unique_word)):
    word_to_idx[word] = idx
    
    
#convert word to ids
def convert_words_to_ids(tokens):
    idx = []
    for token in tokens:
        token_id = word_to_idx.get(token)
        idx.append(token_id)
    return idx
        
    

# assign ids to words
train['comment_text'] = train['comment_text'].apply(convert_words_to_ids)

print(train['comment_text'])
#max_length of sentence
length_of_sequences = []
for i,j in enumerate(train['comment_text']):
    x = len(train['comment_text'][i])
    length_of_sequences.append(x)
max_length = max(length_of_sequences)

#pad the sentences
padded_sentence = []
for i ,sentence in enumerate(train['comment_text']):
    length = len(sentence)
    pad = sentence + [0]*(max_length-length)
    padded_sentence.append(pad)

0      [2852, 2547, 4888, 8805, 3603, 5132, 2957, 701...
1      [2023, 5008, 727, 1540, 3925, 7319, 7924, 8277...
2      [3709, 4937, 3925, 6712, 8566, 2532, 9024, 353...
3      [5298, 1216, 4916, 6705, 8002, 3982, 9262, 730...
4                   [7525, 3699, 1331, 6886, 5873, 8286]
                             ...                        
995    [3711, 9370, 9096, 9164, 3796, 4720, 6141, 978...
996    [1577, 4513, 4432, 4433, 4350, 989, 2190, 4467...
997    [478, 8842, 7041, 2822, 546, 3878, 5340, 6680,...
998    [8566, 4090, 3843, 2782, 9229, 2902, 7062, 584...
999    [8410, 6774, 302, 3560, 9068, 1332, 5268, 4954...
Name: comment_text, Length: 1000, dtype: object


In [15]:
max_length

793

In [16]:
padded_sentence[0]

[2852,
 2547,
 4888,
 8805,
 3603,
 5132,
 2957,
 7017,
 9102,
 8880,
 1498,
 3322,
 8991,
 5485,
 9424,
 2415,
 2900,
 6173,
 2427,
 6896,
 8227,
 8136,
 5873,
 7513,
 3925,
 7000,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0

In [17]:
# Assign the sentences to 
train['comment_text'] = padded_sentence

In [18]:
## CHECK if the words are not padded
for i,j in enumerate(train['comment_text']):
    if len(train['comment_text'][i]) != max_length:
        print(i)

## 3. DEFINE RNN ARCHITECTURE

RNN ARCHITECTURE : INPUT AND OUTPUT

https://towardsdatascience.com/pytorch-basics-how-to-train-your-neural-net-intro-to-rnn-cb6ebc594677#:~:text=torch.-,nn.,batch%2C%20num_directions%20*%20hidden_size)%20.

rnn class

1. **for unbatched** 
input tensor (sequence_length,input_size)
output tensor(sequence_length,D * hidden_size)

D = 2 IF bidirectional =True, otherwise 1


2. **when batch_first = False**
input tensor(sequence_length,batch_size, input_size), 
output tensor(sequence_length,batch_size, D*hidden_size)

when batch_first = True
input tensor(batch_size,sequence_length,input_size)
output tensor(batch_size,sequence_length,D*hidden_size)


3. **Output of rnn is output and h_n**

'output' contains hidden states for each time step of each sequence in the batch
Shape of the output varies according to batch_first parameter

'h_n' contains the final hidden state for each elements in the batch 

for unbatched input
hn is (D*num_layers,hidden_size)
h

In [19]:
# ## design model
# class RNNmodel(nn.Module): 
#     def __init__(self,vocab_size,embedding_dimension,hidden_dim,output_dim):
#         super(RNNmodel,self).__init__()
#         self.embeddinglayer = nn.Embedding(num_embeddings = vocab_size,embedding_dim = embedding_dimension)
#         '''num_embeddings -> size of dictionary of embeddings(total number of unique words or tokens
#         in your vocabulary).
#         embedding_dim -> size of each embedding vector(dimensionality of embeddings)'''
#         self.rnn = nn.RNN(input_size=embedding_dimension,hidden_size=hidden_dim)
#         self.linear = nn.Linear(hidden_dim,output_dim)
#         self.sigmoid = nn.Sigmoid()
        
#     def forward(self,text):
#         embedded_layer = self.embeddinglayer(text)
#         out,hidden  = self.rnn(embedded_layer)
#         output2 = self.linear(hidden)
#             # for classification task we use the output value of all time steps in last rnn
#             ## for other seq 2 seq task we use hidden value of last time step 
#         final = self.sigmoid(output2)
#         return final 

The output of the `self.rnn(embedded)` statement in PyTorch, when using a basic RNN layer, consists of two main components: `output` and `hidden`. Let's break down what each of these represents and their shapes:

### `output`
- `output` contains the hidden states for each time step of each sequence in the batch.
- Its shape varies depending on whether the input was batched and the `batch_first` parameter.

#### Shape of `output`
1. **Unbatched Input (`batch_first=False`)**
   - Shape: `(seq_len, batch_size, num_directions * hidden_size)`
   - Here, `seq_len` is the length of the sequence, `batch_size` is the number of sequences (if batched), `num_directions` is 1 for a simple RNN (2 for bidirectional RNNs), and `hidden_size` is the size of the hidden layer.

2. **Batched Input (`batch_first=True`)**
   - Shape: `(batch_size, seq_len, num_directions * hidden_size)`
   - The same dimensions as above, but the batch size and sequence length dimensions are swapped.

### `hidden`
- `hidden` represents the final hidden state for each element in the batch.
- It's particularly important for understanding the final state of the sequence, which is often used in sequence-to-label tasks (like classification).

#### Shape of `hidden`
- Shape: `(num_layers * num_directions, batch_size, hidden_size)`
- `num_layers` is the number of layers in the RNN, `num_directions` is 1 for a simple RNN (and 2 for bidirectional), `batch_size` is the number of sequences, and `hidden_size` is the size of the hidden layer.

### Explanation
1. **For Each Time Step (`output`):**
   - The RNN processes each element of the sequence one by one. At each time step, it updates its hidden state based on the current input and the previous hidden state.
   - The `output` tensor collects these hidden states for each time step across all sequences in the batch. It's useful if you need to do further processing at each time step (e.g., in sequence-to-sequence models).

2. **Final State of the Sequence (`hidden`):**
   - After processing the last element of the sequence, the RNN's hidden state represents the final encoding of the entire sequence.
   - The `hidden` tensor captures this final state. It's especially useful for tasks like classification where the entire sequence's context needs to be considered for a single prediction.

### In Summary
- `output`: Provides a full sequence of hidden states, useful for tasks where every time step's output is important.
- `hidden`: Provides the final hidden state, useful for tasks where the final state of the sequence is critical (like classification).

The structure and shape of these outputs allow RNNs to be versatile for a variety of sequence processing tasks in PyTorch.

## I was making mistake by not squeezing the hidden dimsion and my output from forward function is coming of size (16,793)

The issue you're encountering seems to be related to the dimensions of the output from your RNN model. When using an RNN for binary classification, the typical approach is to use the last hidden state to make a prediction. However, the shape of the output seems to be inconsistent with what is expected for binary classification.

Let's break down the potential issues and solutions:

1. **Understanding RNN Output:** The `self.rnn` layer in your code returns two outputs: `out` and `hidden`. The `out` contains the hidden states from all timesteps, while `hidden` is the last hidden state. For binary classification, you generally use the last hidden state.

2. **Shape of RNN Output:** The output shape of `hidden` from an RNN in PyTorch is `(num_layers * num_directions, batch, hidden_size)`. If you are using a single layer, unidirectional RNN, this shape becomes `(1, batch_size, hidden_size)`. 

3. **Linear Layer Dimension Issue:** You have defined your linear layer with an output dimension of 2 (`self.linear = nn.Linear(hidden_dim, 2)`). This is suitable for a multi-class classification problem with 2 classes, but for binary classification, you typically have a single output unit which predicts the probability of one class (with the other class's probability being `1 - predicted_probability`). Therefore, your linear layer should have an output dimension of 1 for binary classification.

4. **Squeezing the Hidden State:** Before passing the hidden state to the linear layer, you should squeeze the first dimension (which is 1) to match the input shape expectation of the linear layer. 

5. **Sigmoid Activation:** The sigmoid activation is correctly used for binary classification. However, you should apply it after the linear layer on its output, ensuring that the output is a single value representing the probability of the positive class for each instance in the batch.

Here's how you can modify your `forward` method:

```python
def forward(self, text):
    embedded_layer = self.embeddinglayer(text)
    out, hidden = self.rnn(embedded_layer)
    # hidden is of shape (1, batch_size, hidden_size), squeeze the first dimension
    hidden = hidden.squeeze(0) # Now hidden is of shape (batch_size, hidden_size)
    output2 = self.linear(hidden) # Now output2 is of shape (batch_size, 1)
    final = self.sigmoid(output2)
    return final.squeeze()  # Ensuring the output is of shape (batch_size)
```

And make sure your linear layer is defined for binary classification:

```python
self.linear = nn.Linear(hidden_dim, 1)
```

This should correct the dimensionality issue and produce an output tensor that matches the size of your batch, thus resolving the mismatch error.

In [20]:
## design model
class RNNmodel(nn.Module): 
    def __init__(self,vocab_size,embedding_dimension,hidden_dim,output_dim):
        super(RNNmodel,self).__init__()
        self.embeddinglayer = nn.Embedding(num_embeddings = vocab_size,embedding_dim = embedding_dimension)
        '''num_embeddings -> size of dictionary of embeddings(total number of unique words or tokens
        in your vocabulary).
        embedding_dim -> size of each embedding vector(dimensionality of embeddings)'''
        self.rnn = nn.RNN(input_size=embedding_dimension,hidden_size=hidden_dim,batch_first =True)
        self.linear = nn.Linear(hidden_dim,output_dim)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,text):
        embedded_layer = self.embeddinglayer(text)
        out,hidden  = self.rnn(embedded_layer)
        #print(f"Shape of hidden state before squeeze: {hidden.shape}")
        # hidden is of shape (1,batch_size,hidden_size), will squeeze the first dimesnion
        hidden = hidden.squeeze(0) # now hidden is of shape (batch_size,hidden_size)
        output2 = self.linear(hidden)# output 2 is of shape (batch_size,1)
        # for seq 2 seq task we use the output value of all time steps in last rnn
        ## for classification task we use hidden value of last time step 
        final = self.sigmoid(output2)
        return final.squeeze()

**Mistake : I was not putting batch_first = true in nn.rnn
(
self.rnn = nn.RNN(input_size=embedding_dimension,hidden_size=hidden_dim,batch_first =True)
) 
which caused a error that hidden dimension to be of shape(1,763,256) (batch,sequence_length,hidden_dimesnion), but the hidden state shpuld have a shape of (1,16,256) i.e (1,batch_size,hidden_size) after the rnn**

From your provided code and the issue description, it seems that the problem might not be in the model architecture itself, but rather in how the data is being processed or fed into the model. The output size of `793` is particularly intriguing as it does not directly correspond to a typical dimension in an RNN for binary classification (like batch size or hidden size).

Here are some potential issues to check:

1. **Data Loader and Batch Processing:**
   - Ensure that `X_train` and `y_train` in your training loop are tensors of the correct shape. `X_train` should be a tensor of shape `[batch_size, seq_len]`, where `seq_len` is the length of your sequences, and `y_train` should be of shape `[batch_size]`.
   - In your training loop, you are iterating over `train_loader` but only using the last batch for training and validation. The line `X_train, y_train = batch` and similar for validation should be inside the loop body, not outside.

2. **Input to the Model:**
   - Verify that the input text data (`X_train` and `X_val`) is correctly preprocessed and converted to tensor format suitable for input to your RNN. This includes tokenization, numericalization (converting words to indices), and padding if necessary.

3. **Dimension of Hidden State in RNN:**
   - Double-check that the `hidden_dim` parameter in your RNNmodel class matches the expected size. The hidden state should have a shape of `[1, batch_size, hidden_size]` after the RNN and before squeezing.

4. **Output Dimension of Linear Layer:**
   - For binary classification, the `output_dim` in your model's constructor should be `1`, and it looks like it might be correctly set. Just verify it.

5. **Check Model Output:**
   - After getting `y_pred` from your model, print its shape to confirm it is `[batch_size]`. If it's not, there might be an issue in the forward pass of your model.

6. **Error in Data Preparation or Model Call:**
   - There might be an issue in how the data is being prepared or an inconsistency in how the model is called with the data. Double-check the preprocessing steps.




In [21]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"[2852, 2547, 4888, 8805, 3603, 5132, 2957, 701...",0,0,0,0,0,0
1,000103f0d9cfb60f,"[2023, 5008, 727, 1540, 3925, 7319, 7924, 8277...",0,0,0,0,0,0
2,000113f07ec002fd,"[3709, 4937, 3925, 6712, 8566, 2532, 9024, 353...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"[5298, 1216, 4916, 6705, 8002, 3982, 9262, 730...",0,0,0,0,0,0
4,0001d958c54c6e35,"[7525, 3699, 1331, 6886, 5873, 8286, 0, 0, 0, ...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
995,02b8e9f1f138d728,"[3711, 9370, 9096, 9164, 3796, 4720, 6141, 978...",0,0,0,0,0,0
996,02b90e56ec25a4c1,"[1577, 4513, 4432, 4433, 4350, 989, 2190, 4467...",0,0,0,0,0,0
997,02b91acc085c26f8,"[478, 8842, 7041, 2822, 546, 3878, 5340, 6680,...",0,0,0,0,0,0
998,02b94ce316048bc1,"[8566, 4090, 3843, 2782, 9229, 2902, 7062, 584...",0,0,0,0,0,0


In [22]:
## load into dataloaders
X = train['comment_text']
y = train['toxic'].values
X_tensor = torch.tensor(X,dtype = torch.long)
y_tensor= torch.tensor(y,dtype =torch.float)

dataset = TensorDataset(X_tensor,y_tensor)
# train_dataset = DataLoader(dataset,batch_size=16,shuffle=True)



In [23]:
## USing the train data only as validation data because for validation data the unique words needs tonbe added into vocabulary 

from torch.utils.data import random_split
# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders for the train and test sets
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [24]:
for i,j in train_dataset:
    print(i,j)
    break
## i -> X_tensor

tensor([7747, 2129,  355, 8125, 8112, 6143,  355, 8125, 6945, 7745, 2126, 9164,
        2425, 7300, 1907, 7747, 2129,  546,  446, 6045, 3504, 6013, 4041, 7949,
        5597,  546, 7949, 4004, 2645, 1907, 7747, 2129,  551, 4041, 7951, 3967,
        7496, 5035, 2126, 8394, 6173, 7312, 3524, 3336,   46, 5597, 9099, 7952,
        5595, 3523,  926, 8331, 5606, 6143, 2722, 5035, 1749, 2129,  113, 8442,
        5873, 2818, 7747, 2129, 2034, 8112, 1865,  113, 5599, 8136, 5873, 2849,
        6249,  704, 8113, 7747, 2129,  546, 5073, 1908, 5035, 2126, 9242, 2123,
        6173, 6896, 7747, 2129, 8112, 2427, 3705,  110, 4076,  546, 9303, 9303,
        6902, 1672, 9192, 6206, 3524,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

i -> X_tensor contains tokenized  text data, i is a subset of this tensor corresponding to the batch size.

j -> batch of labels from y_tensor

In [25]:
vocab_size = len(unique_word)
embedding_dimension = 100
hidden_dim = 256
output_dim = 1

model = RNNmodel(vocab_size = vocab_size,embedding_dimension=embedding_dimension,hidden_dim=hidden_dim,output_dim=output_dim)
# model.to(device)


## define a levaluation function
loss_fn = nn.BCELoss()
## optimizer
optimizer = torch.optim.Adam(model.parameters(),
                            lr = 0.001)
# define evaluation 
from sklearn.metrics import roc_auc_score
def roc_score(y_train,y_pred):
    y_train = y_train.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
    roc = roc_auc_score(y_train,y_pred)
    return roc

In [26]:
%%time
## Training

epochs = 1



for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        X_train , y_train = batch
        y_pred = model(X_train)
        y_pred = y_pred.squeeze()
        loss = loss_fn(y_pred,y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        for batches in val_loader:
            X_val , y_val = batches
            pred = model(X_val)
            val_loss = loss_fn(pred,y_val)
            #val_roc = roc_score(y_val,pred)
    
    if epoch%10==0:
        print(f"epoch {epoch} | train_loss {loss} | validation loss {val_loss} ")
        
        

epoch 0 | train_loss 0.38204705715179443 | validation loss 0.3821089565753937 
CPU times: user 18 s, sys: 254 ms, total: 18.3 s
Wall time: 9.25 s


In [27]:
y_pred.shape,y_train.shape

(torch.Size([16]), torch.Size([16]))

In [28]:
X_train.shape

torch.Size([16, 793])

## Count Vectorizer

In [29]:
## create Words into vectors using Count Vectorizer
train = train2

from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer()
## tokenize using count vectorizer
X = cvec.fit_transform(train['comment_text'])
y = train['toxic'].values

# convert to pytorch tensors
from torch.utils.data import TensorDataset, DataLoader
X_tensor = torch.tensor(X.toarray()).float()
y_tensor = torch.tensor(y).long()

## create Tensor Dataset
dataset = TensorDataset(X_tensor,y_tensor)

train_loader = DataLoader(dataset,batch_size=32,shuffle = True)

In [30]:
## same way tfifd , glove can be used 

In [31]:
## use token

In [32]:
## SIMPLE RNN 
## DATASET CLASS
## load data into dataloader
## define architecture
## dfine loss function
## define optimizer 
## train
## evaluate
## predict